In [1]:
# Description: Open Shop Scheduling Problem using Constraint Programming
# import the necessary classes
import pandas as pd
import numpy as np
import time
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model
from ortools.constraint_solver.pywrapcp import IntervalVar 
from ortools.init import pywrapinit
import collections
import pandas as pd 


In [6]:
def extract_data(filename):
    # Initialize variables to store the data.
    nj = []
    nm = []
    times = []
    machines = []

    task_counter = 0
    task_matrix = []


    # Open the file and read each line.
    with open(filename, 'r') as file:
        contents = file.read()
        lines = contents.splitlines()

        for line in range(len(lines)):
            # Check if the line contains the number of jobs and machines.
            if 'number of jobs' in lines[line]:
                nj.append(lines[line + 1].split()[0])
                nm.append(lines[line + 1].split()[1])
                break
        
        num_jobs = int(nj[0])
        num_machines = int(nm[0])
        for line in range(len(lines)):

            if 'number of jobs' in lines[line]:
                continue

            # elif ' ' in lines[line]:
            #     continue

            elif 'processing times :' in lines[line]:

                task_matrix.append(task_counter)
                task_matrix[task_counter] = []

                job_counter = 0

                pos = 0
                while pos < num_jobs:
                    pos += 1
                    time_var = lines[line + pos].split()
                    machine_var = lines[line + pos + num_jobs + 1].split()

                    task_matrix[task_counter].append([])
                    
                    #print('new times/machines pairs')
                    for aux in range(len(time_var)):
                        #print(f'{time_var[aux]} , {machine_var[aux]}')
                        task_matrix[task_counter][job_counter].append((int(time_var[aux]), int(machine_var[aux])))
                    
                    job_counter += 1

                task_counter +=1

        #print(task_matrix)
    
    for i in range(len(task_matrix)):
        for j in range(len(task_matrix[i])):
            for k in range(len(task_matrix[i][j])):
                task_matrix[i][j][k] = tuple(reversed(task_matrix[i][j][k]))

    return (num_jobs, num_machines, task_matrix)

In [7]:
num_jobs, num_machines, smaller_lists = extract_data('tai4_4.txt')

In [8]:
def main(jobs_data):
    """Entry point of the program."""
    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(1,machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # # Precedences inside a job.
    # for job_id, job in enumerate(jobs_data):
    #     for task_id in range(len(job) - 1):
    #         model.Add(all_tasks[job_id, task_id +
    #                             1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 60.0 
    solver.parameters.num_search_workers = 8
    solver.parameters.log_search_progress = True
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_task_%i' % (assigned_task.job,
                                           assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())
    print('  - status   : %s' % solver.StatusName(status))
    print('  - objective: %i' % solver.ObjectiveValue())
    print('  - User Time  : %i' % solver.UserTime())

In [9]:
main(smaller_lists[0])

Solution:
Optimal Schedule Length: 116.0
Machine 1: job_1_task_1   job_0_task_1   job_2_task_0   job_3_task_0   
           [0,15]         [15,49]        [49,87]        [87,182]       
Machine 2: job_0_task_3   job_3_task_2   job_2_task_1   job_1_task_2   
           [0,2]          [2,9]          [9,28]         [28,117]       
Machine 3: job_2_task_2   job_1_task_3   job_3_task_1   job_0_task_0   
           [0,28]         [28,98]        [98,132]       [132,186]      
Machine 4: job_3_task_3   job_2_task_3   job_1_task_0   job_0_task_2   
           [0,29]         [29,116]       [116,125]      [125,186]      


Statistics
  - conflicts: 0
  - branches : 21
  - wall time: 0.029321 s
  - status   : OPTIMAL
  - objective: 116
  - User Time  : 0


In [10]:
list_files = ['tai4_4.txt','tai5_5.txt','tai7_7.txt','tai10_10.txt','tai15_15.txt','tai20_20.txt']
start_cell_time = time.time()
for i in range(len(list_files)):
    start_list_time = time.time()
    print(f'This is the file: {list_files[i]}')
    num_jobs, num_machines, smaller_lists = extract_data(list_files[i])
    for i in range(len(smaller_lists)):
        start_time = time.time()
        print (f'This is the list number: {i}')
        main(smaller_lists[i])
        print("--- %.2f seconds ---" % (time.time() - start_time))
    print("--- %.2f seconds for the file---" % (time.time() - start_list_time))
print("--- %.2f seconds for the cell---" % (time.time() - start_cell_time))



This is the file: tai4_4.txt
This is the list number: 0
Solution:
Optimal Schedule Length: 116.0
Machine 1: job_1_task_1   job_0_task_1   job_2_task_0   job_3_task_0   
           [0,15]         [15,49]        [49,87]        [87,182]       
Machine 2: job_0_task_3   job_3_task_2   job_2_task_1   job_1_task_2   
           [0,2]          [2,9]          [9,28]         [28,117]       
Machine 3: job_2_task_2   job_1_task_3   job_3_task_1   job_0_task_0   
           [0,28]         [28,98]        [98,132]       [132,186]      
Machine 4: job_3_task_3   job_2_task_3   job_1_task_0   job_0_task_2   
           [0,29]         [29,116]       [116,125]      [125,186]      


Statistics
  - conflicts: 0
  - branches : 74
  - wall time: 0.022949 s
  - status   : OPTIMAL
  - objective: 116
  - User Time  : 0
--- 0.03 seconds ---
This is the list number: 1
Solution:
Optimal Schedule Length: 128.0
Machine 1: job_0_task_2   job_3_task_3   job_2_task_0   job_1_task_1   
           [0,45]         [45,1